# Importing datasets
* Converts data obtained form the MyAnimeList API into a shared format
* Note that loading and saving the datasets may take several minutes

In [ ]:
import datetime
import glob
import logging
import os

import pandas as pd

In [ ]:
source_dir = "../../data/mal/"

In [ ]:
outdir = "../../data/raw_data"
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:
# logging
logger = logging.getLogger("MyAnimeList")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    "%(name)s:%(levelname)s:%(asctime)s: %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
)
for stream in [
    logging.StreamHandler(),
]:
    stream.setFormatter(formatter)
    logger.addHandler(stream)

## Get anime facts

In [ ]:
anime = pd.read_csv(os.path.join(source_dir, "anime_facts/anime.csv"))

In [ ]:
anime = anime.sort_values(by="anime_id")

In [ ]:
anime.to_csv(os.path.join(outdir, "anime.csv"), index=False)

In [ ]:
%run ImportMalUserLists.ipynb

In [ ]:
if os.path.exists(os.path.join(outdir, "user_anime_list.mal.csv")):
    os.remove(os.path.join(outdir, "user_anime_list.mal.csv"))

In [ ]:
for file in glob.glob(os.path.join(source_dir, "user_anime_facts/user_anime_list.*.csv")):
    verify_user_anime_list_consistency(file)

In [ ]:
for file in glob.glob(os.path.join(source_dir, "user_anime_facts/user_anime_list.*.csv")):
    process_user_anime_lists(
        os.path.join(source_dir, file),
        os.path.join(outdir, "user_anime_list.mal.csv"),
    )